<a href="https://colab.research.google.com/github/ankit-rathi/DE-with-AWS/blob/main/Try_Kinesis_Data_Stream.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install boto3

In [18]:
from google.colab import drive
drive.mount('/content/drive')
import os

import pandas as pd

project_path = '/content/drive/My Drive/Personal'
os.chdir(project_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
aws_keys_df = pd.read_csv('aws-rootkey.csv')

In [20]:
import boto3
import json
import time

# Replace with your actual AWS Access Key ID and Secret Access Key
AWS_ACCESS_KEY_ID = aws_keys_df['Access_key_ID'][0]
AWS_SECRET_ACCESS_KEY = aws_keys_df['Secret_access_key'][0]
REGION_NAME = aws_keys_df['Region'][0]

# Initialize a session using your credentials
session = boto3.Session(
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name=REGION_NAME
)

# Create clients for different AWS services
kinesis = session.client('kinesis')

In [21]:
# Define the stream name and shard count
stream_name = 'MyKinesisStream'
shard_count = 1

# Create a Kinesis stream
try:
    kinesis.create_stream(StreamName=stream_name, ShardCount=shard_count)
    print(f"Stream {stream_name} created successfully.")
except kinesis.exceptions.ResourceInUseException:
    print(f"Stream {stream_name} already exists.")


Stream MyKinesisStream created successfully.


In [22]:
# Wait for the stream to be active
while True:
    response = kinesis.describe_stream(StreamName=stream_name)
    stream_status = response['StreamDescription']['StreamStatus']
    if stream_status == 'ACTIVE':
        print(f"Stream {stream_name} is active and ready.")
        break
    time.sleep(2)

Stream MyKinesisStream is active and ready.


In [23]:
# Put records into the stream
for i in range(5):
    data = json.dumps({'index': i, 'message': f'This is record {i}'})
    partition_key = str(i)
    kinesis.put_record(
        StreamName=stream_name,
        Data=data,
        PartitionKey=partition_key
    )
    print(f"Put record {i} into stream {stream_name}.")

Put record 0 into stream MyKinesisStream.
Put record 1 into stream MyKinesisStream.
Put record 2 into stream MyKinesisStream.
Put record 3 into stream MyKinesisStream.
Put record 4 into stream MyKinesisStream.


In [24]:
# Get shard iterator
response = kinesis.describe_stream(StreamName=stream_name)
shard_id = response['StreamDescription']['Shards'][0]['ShardId']

shard_iterator_response = kinesis.get_shard_iterator(
    StreamName=stream_name,
    ShardId=shard_id,
    ShardIteratorType='TRIM_HORIZON'
)
shard_iterator = shard_iterator_response['ShardIterator']

# Read records from the stream
record_response = kinesis.get_records(ShardIterator=shard_iterator, Limit=2)
records = record_response['Records']

while 'NextShardIterator' in record_response:
    record_response = kinesis.get_records(ShardIterator=record_response['NextShardIterator'], Limit=2)
    records.extend(record_response['Records'])
    if len(record_response['Records']) == 0:
        break

# Print out the records
for record in records:
    print(record['Data'].decode('utf-8'))


{"index": 0, "message": "This is record 0"}
{"index": 1, "message": "This is record 1"}
{"index": 2, "message": "This is record 2"}
{"index": 3, "message": "This is record 3"}
{"index": 4, "message": "This is record 4"}


In [25]:
# Delete the Kinesis stream
kinesis.delete_stream(StreamName=stream_name, EnforceConsumerDeletion=True)
print(f"Stream {stream_name} deleted.")

Stream MyKinesisStream deleted.
